# Exercise 2: Normalizing the Data (Simplified)

* DS 6001
* Raf Alvarado
* 16 October 2019

# Overview

In this exercise, we split the raw table into tables corresponding the **implied subjects** we discovered. Then we use joins to reassemble the data. Recall that we noticed ARTIST, GENRE, ALBUM, and REVIEW.

To do this, we use Pandas to store the initial table as a dataframe, and then we create variations of that dataframe and save those to the database as new tables. 

# Implied Subjects (E-R  Diagram)

The data looks good. Let's say we are going to be building a data product around albums and reviews, and we want to use Pitchfork data as one component of the's data. In using these data, we should find the **implied subjects** in the dataframe and create separate tables to use as placeholders for more information.

What are the implied subjects? ALBUM, ARTIST, GENRE, and REVIEW.

If we were to create a full blown database, we might create something like this:

<img src="er2.png" style="height:400px;"/>

<h1>FIX THIS<?h1>

In [1]:
import sqlite3
import pandas as pd
%matplotlib inline

# Connect to the database

In [2]:
db = sqlite3.connect('../Lesson_1/pitchfork.db')

# Define convenience function

In [3]:
def q(sql="", params=(), db=db):
    df =  pd.read_sql_query(sql, db, params=params)
    return df

# Get the raw review table

Get the review data, but don't import the records where there is no album title given. (I noticed that some titles were missing when playing with the data in SQLiteStudio.)

In [4]:
sql_raw = """
SELECT * 
FROM review_raw 
WHERE album IS NOT NULL
"""
raw = q(sql_raw).set_index('id')

In [5]:
raw.head()

,album,artist,best,date,genre,review,score
id,,,,,,,
1,A.M./Being There,Wilco,1,December 6 2017,Rock,Best new reissue 1 / 2 Albums Newly reissued a...,7.0
2,No Shame,Hopsin,0,December 6 2017,Rap,"On his corrosive fifth album, the rapper takes...",3.5
3,Material Control,Glassjaw,0,December 6 2017,Rock,"On their first album in 15 years, the Long Isl...",6.6
4,Weighing of the Heart,Nabihah Iqbal,0,December 6 2017,Pop/R&B,"On her debut LP, British producer Nabihah Iqba...",7.7
5,The Visitor,Neil Young / Promise of the Real,0,December 5 2017,Rock,"While still pointedly political, Neil Youngís ...",6.7


In [6]:
raw.shape

(19550, 7)

# Create implied subject tables

## The `artist` table

We get the unique artist names and save them to a new dataframe, and change the column name and the index name in the process so that we have well-named columns in our table. 

In [7]:
sql_artists = """
SELECT artist AS artist_name, count() AS n_reviews 
FROM review_raw
GROUP BY artist
"""
artists = q(sql_artists)
# artists.index.name = 'artist_id'

In [8]:
artists.head()

,artist_name,n_reviews
0,!!!,7
1,!!! / Out Hud,1
2,(Sandy) Alex G,1
3,+/-,6
4,...And You Will Know Us by the Trail of Dead,10


In [9]:
artists.to_sql('artist', db, index=False, if_exists='replace')

## The `genre` table

In [10]:
sql_genre = """
SELECT genre AS genre_name, count() AS n_reviews
FROM review_raw
GROUP BY genre
"""
genres = q(sql_genre)
# genres.index.name = 'genre_id'

In [11]:
genres.head()

,genre_name,n_reviews
0,Electronic,4020
1,Experimental,1699
2,Folk/Country,700
3,Global,178
4,Jazz,257


In [12]:
genres.to_sql('genre', db, index=False, if_exists='replace')

## The `album` table

In [13]:
sql_albums = """
SELECT album AS album_name, artist AS artist_name, best AS is_best, genre AS genre_name, count() AS n_reviews
FROM review_raw
GROUP BY album, artist -- NOTE KEY 
"""
albums = q(sql_albums)
# albums.index.name = 'album_id'

In [14]:
albums.head(10)

,album_name,artist_name,is_best,genre_name,n_reviews
0,None,Gord Downie,0,Rock,1
1,None,Inverloch,0,None,1
2,None,Lee Ranaldo,0,Experimental,1
3,None,Queens of the Stone Age,0,Rock,1
4,None,Yves Tumor,0,Experimental,1
5,"""Cassette"" EP",Viet Cong,0,Rock,1
6,"""Couples""",The Long Blondes,0,Electronic,1
7,"""Evidence"" 12""",Carlos Giffoni,0,Experimental,1
8,"""Fantasy Live 1999""",Tokyo Black Star,0,Electronic,1
9,"""Free Gold!""",Indian Jewelry,0,Experimental,1


In [15]:
albums.to_sql('album', db, index=False, if_exists='replace')

## The `review` table

In [16]:
sql_reviews = """
SELECT DISTINCT id AS review_id, album AS album_name, artist AS artist_name, 
    date AS review_date, review AS review_content, score AS album_score
FROM review_raw
"""
reviews = q(sql_reviews)

In [17]:
reviews.head()

,review_id,album_name,artist_name,review_date,review_content,album_score
0,1,A.M./Being There,Wilco,December 6 2017,Best new reissue 1 / 2 Albums Newly reissued a...,7.0
1,2,No Shame,Hopsin,December 6 2017,"On his corrosive fifth album, the rapper takes...",3.5
2,3,Material Control,Glassjaw,December 6 2017,"On their first album in 15 years, the Long Isl...",6.6
3,4,Weighing of the Heart,Nabihah Iqbal,December 6 2017,"On her debut LP, British producer Nabihah Iqba...",7.7
4,5,The Visitor,Neil Young / Promise of the Real,December 5 2017,"While still pointedly political, Neil Youngís ...",6.7


In [18]:
reviews.to_sql('review', db, index=False, if_exists='replace')

Note how the new `review` table has fewer columns -- we no longer have genre. 

# Use `JOIN` to recreate new raw table

In [19]:
sql_combo =  """
SELECT
    review_id, album_name, album.artist_name, genre_name, 
    album_score, is_best, review_date, review_content
FROM review
LEFT JOIN album USING(album_name, artist_name)
LEFT JOIN artist USING(artist_name)
LEFT JOIN genre USING(genre_name)
"""
combo = q(sql_combo)

In [20]:
reviews.shape[0] - combo.shape[0]

0

In [21]:
combo.sort_index().head() 

,review_id,album_name,artist_name,genre_name,album_score,is_best,review_date,review_content
0,1,A.M./Being There,Wilco,Rock,7.0,1.0,December 6 2017,Best new reissue 1 / 2 Albums Newly reissued a...
1,2,No Shame,Hopsin,Rap,3.5,0.0,December 6 2017,"On his corrosive fifth album, the rapper takes..."
2,3,Material Control,Glassjaw,Rock,6.6,0.0,December 6 2017,"On their first album in 15 years, the Long Isl..."
3,4,Weighing of the Heart,Nabihah Iqbal,Pop/R&B,7.7,0.0,December 6 2017,"On her debut LP, British producer Nabihah Iqba..."
4,5,The Visitor,Neil Young / Promise of the Real,Rock,6.7,0.0,December 5 2017,"While still pointedly political, Neil Youngís ..."


# Create `VIEW` of combo table

Since we anticipate using this combo table alot, we create VIEW of it, which is a virtual table. This way, we can add data to our tables and always have an up-to-date view for analytic and display purposes.

In [22]:
sql_combo_view1 = """
DROP VIEW IF EXISTS review_combo
"""
sql_combo_view2 = """
CREATE VIEW review_combo AS
    SELECT review_id, album_name, artist_name, genre_name, album_score, is_best, review_date, review_content
    FROM review
    LEFT JOIN album USING (album_name, artist_name)
    LEFT JOIN artist USING (artist_name)
    LEFT JOIN genre USING (genre_name)
"""
db.execute(sql_combo_view1)
db.execute(sql_combo_view2)
db.commit()

In [23]:
combo = q('SELECT * FROM review_combo')